#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [19]:
!pip install folium


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


Загрузим данные из csv файла в датафрейм.

In [44]:
df = pd.read_csv('input_data.csv', sep=';')
display(df.head())
display(df.tail())
rows, cols = df.shape
print(f"\nВ файле {rows} строк и {cols} столбцов.")

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0
11358149,2021-12-31,1850000,1,5,1,33.0,7.0,55.164522,61.386448,0,0,454080.0,567218.0,74,2591215.0



В файле 11358150 строк и 15 столбцов.


Отобразим на карте координаты наших построек.

In [33]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [50]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # радиус Земли в километрах
    # преобразуем градусы в радианы
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    # разницы координат
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # формула гаверсинуса
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c

moscow_coords = (55.7558, 37.6173)
spb_coords = (59.9343, 30.3351)

# вычисляем расстояние до Москвы
df['dist_to_moscow'] = df.apply( lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], *moscow_coords), axis=1 )

# вычисляем расстояние до СПб
df['dist_to_spb'] = df.apply( lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], *spb_coords), axis=1 )

# создаём признаки
df['is_Moscow'] = df['dist_to_moscow'] <= 20
df['is_Saint_Peterburg'] = df['dist_to_spb'] <= 20

In [52]:
# фильтруем квартиры по признакам и выводим нужные столбцы
moscow_flats = df.loc[df['is_Moscow'], ['geo_lat', 'geo_lon', 'dist_to_moscow']]
spb_flats = df.loc[df['is_Saint_Peterburg'], ['geo_lat', 'geo_lon', 'dist_to_spb']]
print(f"Квартиры в пределах 20 км от центра Москвы:\n {pd.concat([moscow_flats.head(5), moscow_flats.tail(5)])}\n")
print(f"Квартиры в пределах 20 км от центра Санкт-Петербурга:\n {pd.concat([spb_flats.head(5), spb_flats.tail(5)])}")
df.drop(['dist_to_moscow', 'dist_to_spb'], axis=1, inplace=True, errors='raise')

Квартиры в пределах 20 км от центра Москвы:
             geo_lat    geo_lon  dist_to_moscow
10        55.778026  37.540147        5.422207
14        55.690554  37.863546       17.042267
36        55.702794  37.666153        6.640485
38        55.834958  37.637378        8.891036
108       55.638429  37.652912       13.240485
11358046  55.754047  37.620405        0.275209
11358053  55.653843  37.400248       17.704970
11358104  55.688446  37.634551        7.566943
11358106  55.643519  37.516501       13.991879
11358131  55.693189  37.490775       10.547378

Квартиры в пределах 20 км от центра Санкт-Петербурга:
             geo_lat    geo_lon  dist_to_spb
15        59.851179  30.411657    10.181421
63        60.036889  30.226123    12.917831
99        59.936744  30.251800     4.648250
104       60.049649  30.428597    13.840014
151       59.851179  30.411657    10.181421
11357895  60.029827  30.378274    10.890242
11358063  59.882662  30.475553     9.710069
11358070  59.902131  30.274959

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [53]:
df.drop(['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], axis=1, inplace=False, errors='raise')

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,False,False
...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,0,False,False
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,0,False,False
11358147,2021-12-31,850000,2,2,2,37.0,5.0,0,False,False
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,0,False,False


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: date building_type is_Moscow is_Saint_Peterburg

Числовые: price level levels rooms area kitchen_area 

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [59]:
from sklearn.preprocessing import OneHotEncoder
# Копируем df, чтобы не портить оригинал
df2 = df.copy()

# Приводим date к datetime и извлекаем признаки (год, месяц и день)
df2['date'] = pd.to_datetime(df2['date'], errors='coerce')  # errors='coerce' превратит некорректные в NaT
df2['year'] = df2['date'].dt.year
df2['month'] = df2['date'].dt.month
df2['day'] = df2['date'].dt.day

# Убедимся, что булевы признаки числовые
binary_features = ['is_Moscow', 'is_Saint_Peterburg']
df2[binary_features] = df2[binary_features].astype(int)

# Список числовых признаков
num_features = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'year', 'month', 'day']

# OneHot-кодирование building_type с drop='first' (чтобы избежать мультиколлинеарности)
cat_feature = ['building_type']
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# fit_transform на столбце building_type (вход должен быть 2D)
ohe_arr = ohe.fit_transform(df2[cat_feature])

# имена новых столбцов
ohe_cols = ohe.get_feature_names_out(cat_feature)

# превратим в DataFrame (с теми же индексами)
ohe_df = pd.DataFrame(ohe_arr, columns=ohe_cols, index=df2.index)

# Собираем итоговый DataFrame: числовые + булевы + one-hot
final_df = pd.concat([df2[num_features + binary_features], ohe_df], axis=1)

# Проверка
print("Размер итогового датафрейма:", final_df.shape)
print("Первые строки:")
print(final_df.head())
print("\nПоследние строки:")
print(final_df.tail())

Размер итогового датафрейма: (11358150, 17)
Первые строки:
      price  level  levels  rooms  area  kitchen_area  year  month  day  \
0   2451300     15      31      1  30.3           0.0  2021      1    1   
1   1450000      5       5      1  33.0           6.0  2021      1    1   
2  10700000      4      13      3  85.0          12.0  2021      1    1   
3   3100000      3       5      3  82.0           9.0  2021      1    1   
4   2500000      2       3      1  30.0           9.0  2021      1    1   

   is_Moscow  is_Saint_Peterburg  building_type_1  building_type_2  \
0          0                   0              0.0              0.0   
1          0                   0              0.0              0.0   
2          0                   0              0.0              0.0   
3          0                   0              0.0              0.0   
4          0                   0              0.0              0.0   

   building_type_3  building_type_4  building_type_5  building_type_6

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [60]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

df2 = df.copy()

# Приведение даты к datetime
df2['date'] = pd.to_datetime(df2['date'], errors='coerce')

# Признак: сколько дней прошло с первого наблюдения
min_date = df2['date'].min()
df2['days_from_start'] = (df2['date'] - min_date).dt.days

# Признак: отношение этажа квартиры к количеству этажей
df2['floor_ratio'] = df2.apply(
    lambda row: row['level'] / row['levels'] if row['levels'] and not pd.isna(row['levels']) else np.nan,
    axis=1
)

# Теперь можно удалить колонку date
df2 = df2.drop(columns=['date'])

# Подготовка списков признаков
# Категориальные: building_type, is_Moscow, is_Saint_Peterburg
cat_feature = ['building_type']
binary_features = ['is_Moscow', 'is_Saint_Peterburg']

# Числовые — теперь включают:
num_features = [
    'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
    'days_from_start', 'floor_ratio'
]

# Кодирование building_type через OneHotEncoder с drop='first'
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
ohe_arr = ohe.fit_transform(df2[cat_feature])
ohe_cols = ohe.get_feature_names_out(cat_feature)
ohe_df = pd.DataFrame(ohe_arr, columns=ohe_cols, index=df2.index)

# Собираем полный DataFrame перед масштабированием
df_before_scale = pd.concat([
    df2[num_features + binary_features].copy(),
    ohe_df
], axis=1)

# Масштабируем только числовые признаки через StandardScaler
scaler = StandardScaler()

# fit_transform на подматрице числовых признаков
scaled_nums = scaler.fit_transform(df_before_scale[num_features])
scaled_nums_df = pd.DataFrame(scaled_nums, columns=num_features, index=df_before_scale.index)

# Собираем окончательный DataFrame: заменяем числовые на стандартизированные
# при этом держим бинарные и OHE-признаки без изменений
final_df = pd.concat([
    scaled_nums_df,
    df_before_scale[binary_features],
    df_before_scale[ohe_cols]
], axis=1)

# Проверка результата
print("Форма итогового DataFrame:", final_df.shape)
print("Первые строки:")
print(final_df.head())
print("Статистика (среднее и дисперсия) по числовым признакам после стандартизации:")
print(final_df[num_features].mean(), final_df[num_features].std(ddof=0))


Форма итогового DataFrame: (11358150, 16)
Первые строки:
      price     level    levels     rooms      area  kitchen_area  \
0 -0.021932  1.622769  2.665027 -0.621470 -0.840577      0.082486   
1 -0.026996 -0.270043 -0.936859 -0.621470 -0.741051      0.267565   
2  0.019789 -0.459324  0.171414  1.106234  1.175756      0.452644   
3 -0.018651 -0.648605 -0.936859  1.106234  1.065171      0.360105   
4 -0.021686 -0.837886 -1.213927 -0.621470 -0.851636      0.360105   

   days_from_start  floor_ratio  is_Moscow  is_Saint_Peterburg  \
0        -1.858355    -0.308580      False               False   
1        -1.858355     1.452761      False               False   
2        -1.858355    -0.909807      False               False   
3        -1.858355     0.087722      False               False   
4        -1.858355     0.315229      False               False   

   building_type_1  building_type_2  building_type_3  building_type_4  \
0              0.0              0.0              0.0      

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [2]:
class KNNRegressor:
    """Простой k-NN регрессор (запоминает данные и усредняет значения k ближайших)."""
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = int(n_neighbors)  # число соседей k
        self.metric = metric                 # метрика: 'euclidean'
        self._X = None                       # место для хранения признаков обучения
        self._y = None                       # место для хранения целевых значений

    def fit(self, X, y):
        X = np.asarray(X)                    # приводим к numpy
        y = np.asarray(y)
        self._X = X.copy()                   # сохраняем копии данных
        self._y = y.copy()
        return self                          # для chaining

    def _distance(self, a, b):
        # вычисляет расстояния между каждой строкой матрицы a и вектором b
        if self.metric == 'euclidean':
            return np.linalg.norm(a - b, axis=1)
        else:
            return np.linalg.norm(a - b, axis=1)

    def predict(self, X):
        X = np.asarray(X)
        if self._X is None:
            raise ValueError("Model is not fitted yet")  # защита от вызова до fit
        preds = []
        for x in X:
            dists = self._distance(self._X, x)         # расстояния до всех тренировочных точек
            idx = np.argsort(dists)[:self.n_neighbors] # индексы k ближайших
            preds.append(np.mean(self._y[idx]))        # среднее целевых значений соседей
        return np.array(preds)                          # возвращаем numpy-массив


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [3]:
class LinearRegression:
    """
    Линейная регрессия с оптимизаторами: 'SGD', 'momentum', 'adagrad', и 'normal' (closed-form).
    Оптимизаторы используют градиент MSE.
    """
    def __init__(self,
                 learning_rate=0.01,
                 optimization='SGD',    # 'SGD' | 'momentum' | 'adagrad' | 'normal'
                 momentum=0.9,          # только для momentum
                 epsilon=1e-8,          # для устойчивости (AdaGrad)
                 decay_rate=0.0,        # опциональный decay lr по эпохам
                 max_iter=1000,
                 batch_size=1):         # batch_size=1 -> pure SGD; >1 -> mini-batch
        self.learning_rate = float(learning_rate)
        self.optimization = optimization.lower()
        self.momentum = float(momentum)
        self.epsilon = float(epsilon)
        self.decay_rate = float(decay_rate)
        self.max_iter = int(max_iter)
        self.batch_size = int(batch_size)
        self.weights = None    # shape (n_features, 1)
        self.bias = None       # scalar
        self.history = []      # история MSE по эпохам (для итеративных методов)

    def fit(self, X, y):
        """
        Обучение модели.
        X: array-like (n_samples, n_features)
        y: array-like (n_samples,) или (n_samples,1)
        """
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).reshape(-1, 1)
        n_samples, n_features = X.shape

        # инициализация параметров (без использования theta)
        self.weights = np.zeros((n_features, 1))
        self.bias = 0.0

        # вспомогательные переменные для momentum и adagrad
        v_w = np.zeros_like(self.weights)        # скорость для momentum (веса)
        v_b = 0.0                                # скорость для momentum (bias)
        cache_w = np.zeros_like(self.weights)    # накопитель квадратов градиента для AdaGrad
        cache_b = 0.0

        # если выбран closed-form, вычисляем сразу и возвращаем
        if self.optimization == 'normal':
            X_b = np.hstack([np.ones((n_samples, 1)), X])
            theta = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
            # безопасно извлекаем скаляр bias
            self.bias = float(np.asarray(theta[0]).item())
            self.weights = np.asarray(theta[1:]).reshape(-1, 1)
            return self

        # По эпохам (max_iter — число проходов по датасету)
        lr0 = float(self.learning_rate)
        for epoch in range(self.max_iter):
            # простой decay learning rate
            lr = lr0 * (1.0 / (1.0 + self.decay_rate * epoch)) if self.decay_rate else lr0

            # перемешаем данные для стохастичности
            perm = np.random.permutation(n_samples)
            X_sh = X[perm]
            y_sh = y[perm]

            # пробегаем по мини-батчам (batch_size может быть 1)
            for start in range(0, n_samples, self.batch_size):
                end = min(start + self.batch_size, n_samples)
                X_batch = X_sh[start:end]         # shape (m, n_features)
                y_batch = y_sh[start:end]         # shape (m, 1)
                m = X_batch.shape[0]              # действительный размер батча

                # предсказание батча: (m,1)
                preds = X_batch.dot(self.weights) + self.bias

                # ошибка (m,1)
                error = preds - y_batch

                # градиенты для MSE (усреднённые по батчу)
                grad_w = (2.0 / m) * (X_batch.T.dot(error))   # (n_features,1)
                grad_b = (2.0 / m) * np.sum(error)            # scalar

                # обновления в зависимости от оптимизатора
                if self.optimization == 'sgd':
                    # простой SGD: w <- w - lr * grad_w
                    self.weights -= lr * grad_w
                    self.bias -= lr * grad_b

                elif self.optimization == 'momentum':
                    # Momentum: v = mu * v - lr * grad; w += v
                    v_w = self.momentum * v_w - lr * grad_w
                    v_b = self.momentum * v_b - lr * grad_b
                    self.weights += v_w
                    self.bias += v_b

                elif self.optimization == 'adagrad':
                    # AdaGrad: накопление квадратов градиентов, адаптивный шаг
                    cache_w += grad_w ** 2
                    cache_b += grad_b ** 2
                    adjusted_lr_w = lr / (np.sqrt(cache_w) + self.epsilon)
                    adjusted_lr_b = lr / (np.sqrt(cache_b) + self.epsilon)
                    self.weights -= adjusted_lr_w * grad_w
                    self.bias -= float(adjusted_lr_b * grad_b)

                else:
                    raise ValueError(f"Unknown optimization method: {self.optimization}")

            # Запись MSE по эпохе для мониторинга
            train_preds = X.dot(self.weights) + self.bias
            mse = np.mean((train_preds.reshape(-1) - y.reshape(-1))**2)
            self.history.append(mse)

        return self

    def predict(self, X):
        """
        Предсказание: возвращает 1D массив прогнозов.
        """
        X = np.asarray(X, dtype=float)
        if self.weights is None:
            raise ValueError("Model is not fitted yet")
        if X.ndim == 1:
            X = X.reshape(1, -1)
        if X.shape[1] != self.weights.shape[0]:
            raise ValueError(f"Feature dimension mismatch: {X.shape[1]} vs {self.weights.shape[0]}")
        preds = X.dot(self.weights) + self.bias
        return preds.reshape(-1)


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [4]:
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor as SKKNN
from sklearn.linear_model import LinearRegression as SKLinear
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1) Получение исходных данных
data = pd.read_csv('input_data.csv', sep=';')


data = data.head(10000)


# 2) Простая предобработка: date->days, floor_ratio (векторно), OHE для building_type, стандартизация числовых
data = data.copy()
if 'date' in data.columns:
    data['date'] = pd.to_datetime(data['date'], errors='coerce')
    data['days_from_start'] = (data['date'] - data['date'].min()).dt.days
else:
    data['days_from_start'] = 0

# векторизированное и безопасное вычисление floor_ratio (быстро и корректно обрабатывает NaN/0)
if ('level' in data.columns) and ('levels' in data.columns):
    level = pd.to_numeric(data['level'], errors='coerce').astype(float)
    levels = pd.to_numeric(data['levels'], errors='coerce').astype(float)
    ok = levels.notna() & (levels != 0)
    data['floor_ratio'] = 0.0
    data.loc[ok, 'floor_ratio'] = (level[ok] / levels[ok]).values
else:
    data['floor_ratio'] = 0.0

cat_cols = [c for c in ['building_type'] if c in data.columns]
num_cols = [c for c in ['level','levels','rooms','area','kitchen_area','days_from_start','floor_ratio'] if c in data.columns]

# OHE (поддержка разных версий sklearn)
ohe_df = pd.DataFrame(index=data.index)
if cat_cols:
    try:
        ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    except TypeError:
        ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')
    ohe_arr = ohe.fit_transform(data[cat_cols])
    try:
        ohe_cols = ohe.get_feature_names_out(cat_cols)
    except Exception:
        ohe_cols = [f"{cat_cols[0]}_{i}" for i in range(ohe_arr.shape[1])]
    ohe_df = pd.DataFrame(ohe_arr, columns=ohe_cols, index=data.index)

scaler = StandardScaler()
nums_df = pd.DataFrame(scaler.fit_transform(data[num_cols]), columns=num_cols, index=data.index) if num_cols else pd.DataFrame(index=data.index)

X = pd.concat([nums_df, ohe_df], axis=1).fillna(0)
if 'price' not in data.columns:
    raise RuntimeError("Нет целевой колонки 'price' в данных.")
y = data['price'].values

# 3) Разделение выборок 0.8 / 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4) Обучение и прогнозы
results = {}

# KNN: наш
knn_custom = KNNRegressor(n_neighbors=5)
knn_custom.fit(X_train.values, y_train)
y_knn_custom = knn_custom.predict(X_test.values)

# KNN: sklearn
knn_sk = SKKNN(n_neighbors=5)
knn_sk.fit(X_train, y_train)
y_knn_sk = knn_sk.predict(X_test)

# Linear: наш
lr_custom = LinearRegression(optimization='normal')
lr_custom.fit(X_train.values, y_train)
y_lr_custom = lr_custom.predict(X_test.values)

# Linear: sklearn
lr_sk = SKLinear()
lr_sk.fit(X_train, y_train)
y_lr_sk = lr_sk.predict(X_test)

# 5) Метрики: MSE, MAE, RMSE
def metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mse)
    return mse, mae, rmse

results['knn_custom'] = metrics(y_test, y_knn_custom)
results['knn_sklearn'] = metrics(y_test, y_knn_sk)
results['lr_custom'] = metrics(y_test, y_lr_custom)
results['lr_sklearn'] = metrics(y_test, y_lr_sk)

# 6) Вывод таблицы
res_df = pd.DataFrame(results, index=['MSE','MAE','RMSE']).T
print("Результаты (train/test split 0.8/0.2):")
display(res_df.round(3))


Результаты (train/test split 0.8/0.2):


,MSE,MAE,RMSE
knn_custom,9.295231e+15,4945234.605,9.641178e+07
knn_sklearn,9.295231e+15,4945210.573,9.641178e+07
lr_custom,9.276764e+15,6031480.332,9.631596e+07
lr_sklearn,9.276764e+15,6031480.332,9.631596e+07


In [ ]:
Ограничили выборку 10000, так как на полной алгоритм работает очень долго